# Fine-tuning : RF-DETR

Objectif : faire de RF-DETR un détecteur de poissons sur des images de 1920\*1080 pixels

In [1]:
# Importations
import os
import numpy as np
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
import cv2
import matplotlib.pyplot as plt
import pycocotools.coco as coco
from PIL import Image
from rfdetr import RFDETRLarge, RFDETRBase  # Assurez-vous que le package rf-detr est bien installé


# Vérification de la disponibilité du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device utilisé :", device)


Device utilisé : cuda


In [2]:
data_dir = "/home/mateo/Travail/Fine-tune/Dataset/Roboflow"  # chemin vers votre jeu de données
#images_dir = os.path.join(data_dir, "luderick-seagrass/train")
#annotations_file = os.path.join(images_dir, "_annotations.coco.json")

# Charger les annotations COCO
#coco_dataset = coco.COCO(annotations_file)
#print("Nombre d'images dans le dataset :", len(coco_dataset.imgs))


In [3]:
# Cellule 6 : Charger et adapter le modèle rf-detr pré-entraîné
# -----------------------------------------------------
# Nous allons charger le modèle rf-detr du dépôt et le fine-tuner
# Vous pouvez soit utiliser le modèle préentraîné fourni par rf-detr (si disponible) ou initialiser un modèle de DETR et le charger
# Vous pouvez adapter le nombre de classes. Ici, nous avons 1 classe (poisson) + 1 classe pour le fond si nécessaire.
num_classes = 1  # par exemple, 1 classe pour "poisson" et 1 classe pour le "fond"

# Chargement du modèle pré-entraîné (précisez éventuellement le checkpoint préentraîné)
model = RFDETRBase(num_classes=num_classes, pretrained=True)
#model.to(device)
print("Modèle chargé sur l'appareil :", device)


Loading pretrain weights


num_classes mismatch: pretrain weights has 90 classes, but your model has 1 classes
reinitializing detection head with 90 classes


Modèle chargé sur l'appareil : cuda


In [ ]:
model.train(dataset_dir=os.path.join(data_dir, "Fish.v44i.coco"), epochs=15, batch_size=1, grad_accum_steps=1, lr=1e-4)

TensorBoard logging initialized. To monitor logs, use 'tensorboard --logdir output' and open http://localhost:6006/ in browser.
Not using distributed mode
git:
  sha: N/A, status: clean, branch: N/A

Namespace(num_classes=1, grad_accum_steps=1, amp=True, lr=0.0001, lr_encoder=0.00015, batch_size=1, weight_decay=0.0001, epochs=15, lr_drop=100, clip_max_norm=0.1, lr_vit_layer_decay=0.8, lr_component_decay=0.7, do_benchmark=False, dropout=0, drop_path=0.0, drop_mode='standard', drop_schedule='constant', cutoff_epoch=0, pretrained_encoder=None, pretrain_weights='rf-detr-base.pth', pretrain_exclude_keys=None, pretrain_keys_modify_to_load=None, pretrained_distiller=None, encoder='dinov2_windowed_small', vit_encoder_num_layers=12, window_block_indexes=None, position_embedding='sine', out_feature_indexes=[2, 5, 8, 11], freeze_encoder=False, layer_norm=True, rms_norm=False, backbone_lora=False, force_no_pretrain=False, dec_layers=3, dim_feedforward=2048, hidden_dim=256, sa_nheads=8, ca_nheads=1

fatal: not a git repository (or any of the parent directories): .git


Get benchmark
Start training
Grad accum steps:  1
Total batch size:  1
LENGTH OF DATA LOADER: 476


Epoch: [0]  [  0/476]  eta: 0:09:52  lr: 0.000100  class_error: 100.00  loss: 7.7516 (7.7516)  loss_ce: 1.1628 (1.1628)  loss_bbox: 0.3373 (0.3373)  loss_giou: 0.1910 (0.1910)  loss_ce_0: 1.2347 (1.2347)  loss_bbox_0: 0.4709 (0.4709)  loss_giou_0: 0.2525 (0.2525)  loss_ce_1: 1.3176 (1.3176)  loss_bbox_1: 0.3625 (0.3625)  loss_giou_1: 0.1977 (0.1977)  loss_ce_enc: 1.0696 (1.0696)  loss_bbox_enc: 0.7610 (0.7610)  loss_giou_enc: 0.3940 (0.3940)  loss_ce_unscaled: 1.1628 (1.1628)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.0675 (0.0675)  loss_giou_unscaled: 0.0955 (0.0955)  cardinality_error_unscaled: 3899.0000 (3899.0000)  loss_ce_0_unscaled: 1.2347 (1.2347)  loss_bbox_0_unscaled: 0.0942 (0.0942)  loss_giou_0_unscaled: 0.1262 (0.1262)  cardinality_error_0_unscaled: 3899.0000 (3899.0000)  loss_ce_1_unscaled: 1.3176 (1.3176)  loss_bbox_1_unscaled: 0.0725 (0.0725)  loss_giou_1_unscaled: 0.0988 (0.0988)  cardinality_error_1_unscaled: 3899.0000 (3899.0000)  loss_ce_enc_un

Epoch: [0]  [ 70/476]  eta: 0:04:23  lr: 0.000100  class_error: 0.00  loss: 7.4042 (7.5823)  loss_ce: 0.6889 (0.9733)  loss_bbox: 0.5445 (0.4789)  loss_giou: 0.3422 (0.2956)  loss_ce_0: 0.9684 (1.1564)  loss_bbox_0: 0.6201 (0.4877)  loss_giou_0: 0.3497 (0.2974)  loss_ce_1: 0.7718 (1.0169)  loss_bbox_1: 0.6091 (0.4853)  loss_giou_1: 0.3400 (0.2909)  loss_ce_enc: 1.1748 (1.1458)  loss_bbox_enc: 0.5357 (0.5881)  loss_giou_enc: 0.3515 (0.3659)  loss_ce_unscaled: 0.6889 (0.9733)  class_error_unscaled: 0.0000 (30.2095)  loss_bbox_unscaled: 0.1089 (0.0958)  loss_giou_unscaled: 0.1711 (0.1478)  cardinality_error_unscaled: 3899.0000 (3898.1549)  loss_ce_0_unscaled: 0.9684 (1.1564)  loss_bbox_0_unscaled: 0.1240 (0.0975)  loss_giou_0_unscaled: 0.1748 (0.1487)  cardinality_error_0_unscaled: 3899.0000 (3898.0282)  loss_ce_1_unscaled: 0.7718 (1.0169)  loss_bbox_1_unscaled: 0.1218 (0.0971)  loss_giou_1_unscaled: 0.1700 (0.1455)  cardinality_error_1_unscaled: 3899.0000 (3897.8169)  loss_ce_enc_unscale

In [ ]:
Image.open("output/metrics_plot.png")
# Affiche un résumé rapide du modèle
print(model)

In [ ]:
import supervision as sv

ds = sv.DetectionDataset.from_coco(
    images_directory_path=os.path.join(data_dir, "Fish.v44i.coco", "test"),
    annotations_path=os.path.join(data_dir, "Fish.v44i.coco", "test","_annotations.coco.json")
)

In [ ]:
path, image, annotations = ds[38]
image = Image.open(path)

detections = model.predict(image, threshold=0.3)

text_scale = sv.calculate_optimal_text_scale(resolution_wh=image.size)
thickness = sv.calculate_optimal_line_thickness(resolution_wh=image.size)

bbox_annotator = sv.BoxAnnotator(thickness=thickness)
label_annotator = sv.LabelAnnotator(
    text_color=sv.Color.BLACK,
    text_scale=text_scale,
    text_thickness=thickness,
    smart_position=True)

annotations_labels = [
    f"{ds.classes[class_id]}"
    for class_id
    in annotations.class_id
]

detections_labels = [
    f"{ds.classes[class_id]} {confidence:.2f}"
    for class_id, confidence
    in zip(detections.class_id, detections.confidence)
]

annotation_image = image.copy()
annotation_image = bbox_annotator.annotate(annotation_image, annotations)
annotation_image = label_annotator.annotate(annotation_image, annotations, annotations_labels)

detections_image = image.copy()
detections_image = bbox_annotator.annotate(detections_image, detections)
detections_image = label_annotator.annotate(detections_image, detections, detections_labels)

sv.plot_images_grid(images=[annotation_image, detections_image], grid_size=(1, 2), titles=["Annotation", "Detection"])